In [ ]:
import torch
print(torch.cuda.is_available())
print("Done!")

True
Done!


In [ ]:
ADAM_EPS = "adam_eps"
ADAM_OPT = 'adam'
BETA_ONE = "beta_one"
BETA_TWO = "beta_two"
BIDIRECTIONAL = "bideractional"
DROP_WORD = "drop_word"
HIDDEN_DIM = "hidden_dim"
MAX_TRAIN_LENGTH = "max_train_length"
MINI_BATCH_SIZE = "mini_batch_size"
NUM_LAYERS = "num_layers"
ADADELATA_OPT = 'adadelta'
ADAM_OPT = 'adam'
BATCH_SIZE = "batch_size"
BCE_LOSS = 'bce_loss'
CNN_KERNELS = "cnn_kernels"
CNN_OUT_CHANNELS = "cnn_out_channels"
CROSS_ENTROP_LOSS = 'cross_entropy_loss'
DROPOUT = "dropout"
EARLY_STOP_MAX_NO_IMP = "early_stop_max_no_imp"
EMBED_WORDS_FILE = "embed_words_file"
FREEZE_EMBEDDING = "freeze_embeding"
LABEL_COL = "label_col"
LEARNING_RATE = "learning_rate"
LENGTH_COL = "length_col"
LOG_FILE_NAME = "log_file_name"
LOSS_FUNCTION = "loss_function"
MAX_VALID_LOSS = "max_valid_loss"
MIN_FRAME_ITEMS = "min_frame_items"
MIN_VALID_EPOCHS = "min_valid_epochs"
NORM_EMBED_VECS = "norm_embed_vecs"
NUM_LENGTH_FRAMES = "num_length_frames"
OPTIMIZER_NAME = "optimizer_func"
PAD_LABEL = "pad_label"
RANDOM_SAMPLING = "random_sampling"
SEQUENTIAL_SAMPLING = "sequential_sampling"
SGD_OPT = 'sgd'
TEST_SET = "test_set"
TRAIN_SET = "train_set"
SAMPLING_TYPE = "sampling_type"
SEED_VALUE = "seed_value"
TEXT_COL = "text_col"
TOKENS_COL = "tokens_col"
UNK_LABEL = "unk_label"
VALIDATION_SET = "validation_set"

In [ ]:
import json
import numpy as np
from tqdm import tqdm
with open("workshop/nb_config_lstm.json", 'r', encoding='utf-8') as f:
    config = json.load(f)
print(config)

added_words = [PAD_LABEL, UNK_LABEL]
class Embedded_Words:
    def __init__(self, model_file: str, added_pads: list, norm: bool) -> None:
        self.vectors, self.w2i, self.i2w = self.read_model(model_file, added_pads, norm)

    def read_model(self, model_file: str, added_pads: list, norm: bool) -> tuple:
        with open(model_file, "r", encoding="utf-8") as f:
            lines = [x.strip() for x in f.readlines()]

        print(model_file)
        print(len(lines))
        print(lines[0])

        num_word, dim = [int(x) for x in lines[0].split()]
        vectors = np.zeros((num_word + len(added_pads), dim))
        w2i = {}
        i2w = {}
        for line in tqdm(lines[1:]):
            tokens = line.split()
            word = tokens[0]
            word_index = len(w2i)
            v = np.array([float(x) for x in tokens[1:]])
            if norm:
                v = v / np.linalg.norm(v)
            vectors[word_index] = v
            w2i[word] = word_index
            i2w[word_index] = word

        for word in added_pads:
            word_index = len(w2i)
            w2i[word] = word_index
            i2w[word_index] = word
        
        return vectors, w2i, i2w

model_file = "drive/MyDrive/ColabData/embed.model"
embedded_words = Embedded_Words(model_file, added_words, True)
print("")
print(embedded_words.vectors.shape)
print(embedded_words.vectors[embedded_words.vectors.shape[0]-5:,:10])

{'adam_eps': 1e-08, 'batch_first': 1, 'batch_size': 64000, 'beta_one': 0, 'beta_two': 0.98, 'bideractional': 1, 'dropout': 0.5, 'drop_word': 0.1, 'early_stop_max_no_imp': 3, 'embed_words_file': 'workshop/embed.model', 'freeze_embeding': 1, 'hidden_dim': 512, 'learning_rate': 0.001, 'loss_function': 'cross_entropy_loss', 'log_file_name': 'workshop/lstm_nb_init.txt', 'min_valid_epochs': 10, 'max_train_length': 512, 'max_valid_loss': 0.145, 'mini_batch_size': 3000, 'min_frame_items': 500, 'norm_embed_vecs': 1, 'num_epochs': 100, 'num_layers': 1, 'num_length_frames': 12, 'optimizer_func': 'adam', 'rho': 0.95, 'seed_value': -1, 'sampling_type': 'sequential_sampling', 'label_col': 'sentiment', 'length_col': 'lengths', 'text_col': 'review', 'tokens_col': 'tokens', 'train_set': 'workshop/imdb_train_set.csv', 'test_set': 'workshop/imdb_test.csv', 'validation_set': 'workshop/imdb_val_set.csv'}
drive/MyDrive/ColabData/embed.model
85899
85898 256


100%|██████████| 85898/85898 [00:08<00:00, 10363.43it/s]


(85900, 256)
[[ 0.06298662 -0.03220133 -0.02866318  0.12042718  0.18225189 -0.06114065
   0.05848268 -0.04147663  0.04435284 -0.00816747]
 [ 0.05723497 -0.02765993  0.0106675   0.13086651  0.09357034 -0.08705442
  -0.00878455 -0.0667807  -0.00667503 -0.00585762]
 [-0.05147354 -0.00427562  0.09155177  0.13254647  0.11083994 -0.00598478
   0.02139013 -0.03059529 -0.08231317  0.02991129]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]]


In [ ]:
def get_active_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
active_device = get_active_device()
print(active_device)

cuda


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

BATCH_FIRST = True

class LSTM(nn.Module):
    def __init__(self,
                 config: dict,
                 num_classes: int):
        # Constructor.
        super(LSTM, self).__init__()
        
        print('Freeze embedding matrix = ' + str(config[FREEZE_EMBEDDING]))
        print('load embedding model')
        added_words = [PAD_LABEL, UNK_LABEL]
        self.w2v_model = Embedded_Words(model_file, added_words, config[NORM_EMBED_VECS])
        print('\tw2v after padding: ' + str(self.w2v_model.vectors.shape))

        print('generate embedding tensor')
        # Set the embedding module.
        self.embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(self.w2v_model.vectors).to(active_device),
            padding_idx = self.w2v_model.w2i[PAD_LABEL],
            freeze=True) #config[FREEZE_EMBEDDING])
        
        # LSTM layer.
        hidden_dim = config[HIDDEN_DIM]
        bidirectional = bool(config[BIDIRECTIONAL])
        self.lstm = nn.LSTM(self.w2v_model.vectors.shape[1],
                            hidden_dim,
                            num_layers=config[NUM_LAYERS],
                            bidirectional=bidirectional,
                            dropout=0,
                            batch_first=BATCH_FIRST).to(active_device)
        
        # Set the dropout for the embedding layer and the lstm's output layer.
        self.dropout = nn.Dropout(config[DROPOUT])

        # Set the last layer, fully connected.
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes).to(active_device)
        
    # The forward function.
    def forward(self, texts, lengths):
        # Get the embedding of the given text.
        # text = [#batch size, sentence length]
        #lengths = [#batch size]
        embed_text = self.dropout(self.embedding(texts))
        # embed_text = [#batch size, sentence length, embed dim]
        
        # Get the packed sentences.
        packed_text = pack_padded_sequence(embed_text, lengths, batch_first=BATCH_FIRST)
        # packed_text = [[sum lengths, embed dim], [#sequence length (#active batch items for ech length)]]

        # Call the LSTM layer.
        packed_output, (hidden, cell) = self.lstm(packed_text)
        # packed output = [[sum lengths, hidden dim], [#sequence length]]
        # hidden = [1 (2 if bidirectional), #batch size, hidden dim]
        # cell   = [1 (2 if bidirectional), #batch size, hidden dim]
        
        # unpack the output.
        pad_packed_output, _ = pad_packed_sequence(packed_output, batch_first=BATCH_FIRST)
        # pad_packed_output = [batch size, sentence length, hidden dim * 2]
        
        # Prmute the output before pooling.
        permuted_output = self.dropout(pad_packed_output.permute(0, 2, 1))
        # permuted_output = [batch size, hidden dim * 2, sentence length]

        # Max pooling layer.        
        pooled_output = F.max_pool1d(permuted_output, kernel_size=permuted_output.shape[2])
        # pooled_output = [batch size, hidden dim * 2, 1]
        
        # Call the linear full connected layer, after droping out.
        logits = self.fc(torch.squeeze(pooled_output, dim=2))
        # logits = [batch size, #classes]
        
        return logits


In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import TensorDataset
import pandas as pd
import copy
import random

def break_by_batch_size(df: pd.DataFrame, config: dict) -> list:
    sorted_df = df.sort_values(by=config[LENGTH_COL], axis=0, ascending=False, ignore_index=True)
    tokens = sorted_df[config[TOKENS_COL]].to_list()
    labels = sorted_df[config[LABEL_COL]].to_list()
    lengths = sorted_df[config[LENGTH_COL]].to_list()
    
    df_list = []
    batch_size = config[MINI_BATCH_SIZE]
    max_valid_length = config[MAX_TRAIN_LENGTH]
    header = {config[TOKENS_COL]:[], config[LABEL_COL]:[], config[LENGTH_COL]:[]}
    row_index = 0
    num_rows = len(labels)
    while row_index < num_rows:
        num_words = 0
        curr_df = copy.deepcopy(header)
        while num_words < batch_size and row_index < num_rows:
            actual_length = min(max_valid_length, lengths[row_index])
            num_words += actual_length
            curr_df[config[TOKENS_COL]].append(tokens[row_index][:max_valid_length])
            curr_df[config[LABEL_COL]].append(labels[row_index])
            curr_df[config[LENGTH_COL]].append(actual_length)
            row_index += 1
        
        df_list.append(pd.DataFrame(curr_df))
        
    return df_list

DROP_WORD_PROB = -1
def df_to_dataloader(df: pd.DataFrame, w2v_model: Embedded_Words, config: dict, sampling_type: str) -> DataLoader:
    sorted_df = df.sort_values(by=config[LENGTH_COL], axis=0, ascending=False, ignore_index=True)
    texts = sorted_df[config[TOKENS_COL]].values.tolist()
    labels = sorted_df[config[LABEL_COL]].values.tolist()
    lengths = sorted_df[config[LENGTH_COL]].to_list()
    max_len = sorted_df[config[TOKENS_COL]].map(len).max()

    indexed_texts = []
    for sentence in texts:
        sentence += [PAD_LABEL] * (max_len - len(sentence))
        ids = []
        for word in sentence:
            if word not in w2v_model.w2i:
                ids.append(w2v_model.w2i[UNK_LABEL])
            elif np.random.random() < DROP_WORD_PROB:
                ids.append(w2v_model.w2i[UNK_LABEL])
            else:
                ids.append(w2v_model.w2i[word])
        
        indexed_texts.append(ids)
        
    inputs, labels, lengths = tuple(torch.tensor(data) for data in [indexed_texts, labels, lengths])

    data = TensorDataset(inputs, labels, lengths)
    
    if sampling_type == RANDOM_SAMPLING:
        sampler = RandomSampler(data)
    elif sampling_type == SEQUENTIAL_SAMPLING:
        sampler = SequentialSampler(data)
    else:
        print('Wrong Sampling Type: ' + sampling_type)
        return None
        
    dataloader = DataLoader(data, sampler=sampler, batch_size=config[BATCH_SIZE])
    return dataloader
    
def get_data_loaders(input_df: pd.DataFrame,
                     w2v_model: Embedded_Words,
                     config: dict, 
                     sampling_type: str,
                     break_df_func) -> list:
    input_df[config[TOKENS_COL]] = input_df[config[TEXT_COL]].apply(lambda x: x.split(' '))
    input_df[config[LENGTH_COL]] = input_df[config[TOKENS_COL]].map(len)
    df_list = break_df_func(input_df, config)
    dataloaders = []
    for df in df_list:
        dataloader = df_to_dataloader(df, w2v_model, config, sampling_type)
        dataloaders.append(dataloader)
        
    return dataloaders

RHO = 0.95
LEARNING_RATE = 0.001
OPT_NAME = ADAM_OPT
BETA_ONE = 0
BETA_TWO = 0.98
ADAM_EPS = 0.00000001
def get_optimizer(parameters):
    optimizer = None
    if OPT_NAME == ADADELATA_OPT:
        optimizer = optim.Adadelta(parameters,
                                   lr=LEARNING_RATE,
                                   rho=RHO)
    elif OPT_NAME == SGD_OPT:
        optimizer = optim.SGD(parameters, LEARNING_RATE)
    elif OPT_NAME == ADAM_OPT:
        optimizer = optim.Adam(parameters,
                               lr=LEARNING_RATE,
                               betas=(BETA_ONE,BETA_TWO,),
                               eps=ADAM_EPS)
    else:
        print('Wrong optimizer name: ' + OPT_NAME)
        
    return optimizer
    
    
def get_loss_function(func_name: str):
    loss_func = None
    if func_name == CROSS_ENTROP_LOSS:
        loss_func = nn.CrossEntropyLoss()
    elif func_name == BCE_LOSS:
        loss_func = nn.BCELoss()
    else:
        print('Wrong loss function name: ' + func_name)
        
    return loss_func


def set_seed(seed_value: int):
    if seed_value >= 0:
        random.seed(seed_value)
        np.random.seed(seed_value)
        torch.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)


In [ ]:
import time
import torch
import torch.nn as nn

def test(model: nn.Module, dataloaders: list):
    corrects = 0
    evaluated = 0
    start_time = time.time()
    model.eval()
    for dl in dataloaders:
        for texts, labels, lengths in dl:
            texts = texts.to(active_device)
            labels = labels.to(active_device)
            lengths = lengths.to(torch.device("cpu"))

            with torch.no_grad():
                logits = model(texts, lengths)
            preds = torch.argmax(logits, dim=1)
            corrects += (preds == labels).sum().item()
            evaluated += texts.shape[0]
        
    accuracy = corrects / evaluated
    run_time = time.time() - start_time
    return accuracy, run_time


In [ ]:
def end_train(last_model: nn.Module, opt_model: nn.Module, test_dl: list, val_dl: list, config: dict, log_file):
    accuracy, run_time = test(last_model, test_dl)
    str_acc = "{:.5f}".format(accuracy)
    str_time = "{:.1f}".format(run_time)
    log_file.write('Last Model\t' + str_acc + '\t' + str_time + '\n')
    log_file.flush()
    print('Last Model\t' + str_acc + '\t' + str_time)
        
    # Print optimal
    opt_acc, run_time = test(opt_model, test_dl)
    val_acc, run_time = test(opt_model, val_dl)

    test_acc = "test: {:.5f}".format(opt_acc)
    val_acc = "val: {:.5f}".format(val_acc)
    log_file.write('Optimal Model\tTest=' + test_acc + '\tVal=' + val_acc + '\n')
    log_file.flush()
    print('Optimal Model\tTest=' + test_acc + '\tVal=' + val_acc)
    log_file.write("\nConfiguraton:\n")
    sorted_params = sorted(config.items())
    for param in sorted_params:
        log_file.write(param[0] + "\t" + str(param[1]) + "\n")
        log_file.flush()


In [ ]:
import random
def set_seed(seed_value: int):
    if seed_value >= 0:
        random.seed(seed_value)
        np.random.seed(seed_value)
        torch.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)

import time
import copy
import torch.nn as nn
import random
import pandas as pd
NUM_EPOCHS = 40
MAX_NO_IMP = 3
MAX_VALID_LOSS = 0.145
class Trainer:
    def __init__(self):
        pass

    def train(self, config: dict) -> tuple:
        print('latm trainer - start')
        log_file = open(config[LOG_FILE_NAME], "w", encoding="utf-8")
        seed_value = config[SEED_VALUE]
        set_seed(seed_value)
            
        train_df = pd.read_csv(config[TRAIN_SET])
        val_df = pd.read_csv(config[VALIDATION_SET])
        test_df = pd.read_csv(config[TEST_SET])

        pending_model = LSTM(config=config, num_classes=train_df[config[LABEL_COL]].unique().shape[0])
        optimal_model = None
        
        optimizer =  get_optimizer(pending_model.parameters())
        loss_func = nn.NLLLoss()
        log_soft_max = nn.LogSoftmax(dim=1)
        
        num_epochs = NUM_EPOCHS
        print('start training loops. #epochs = ' + str(num_epochs))
        print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Acc':^11} | {'Test Acc':^9} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*50)  
        
        log_file.write(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Acc':^11} | {'Test Acc':^9} | {'Val Acc':^9} | {'Elapsed':^9}\n")
        log_file.write("-"*50 + "\n")
            
        
        best_val_acc = 0
        best_test_acc = 0
        best_test_epoch = -1
        best_epoch = -1
        min_loss = 100
        num_no_imp = 0
        validation_dl = get_data_loaders(val_df, pending_model.w2v_model, config=config, sampling_type=SEQUENTIAL_SAMPLING, break_df_func=break_by_batch_size)
        test_dl = get_data_loaders(test_df, pending_model.w2v_model, config=config, sampling_type=SEQUENTIAL_SAMPLING, break_df_func=break_by_batch_size)
        for i in range(num_epochs):
            epoch = i + 1
            epoch_start_time = time.time()
            total_loss = 0
            num_batches = 0

            train_dl = get_data_loaders(train_df, pending_model.w2v_model, config=config, sampling_type=SEQUENTIAL_SAMPLING, break_df_func=break_by_batch_size)
            random.shuffle(train_dl)
            pending_model.train()
            for dl in train_dl:
                for texts, labels, lengths in dl:
                    texts = texts.to(active_device)
                    labels = labels.to(active_device)
                    lengths = lengths.to(torch.device("cpu"))

                    optimizer.zero_grad()
                    logits = pending_model(texts, lengths)
                    weights = log_soft_max(logits)
                    loss = loss_func(weights, labels)
                    total_loss += loss.item()
                    num_batches += 1
                    loss.backward()
                    optimizer.step()
                
            avg_loss = total_loss / num_batches
            epoch_time = time.time() - epoch_start_time
            
            # Validation test.
            val_acc, val_time = test(pending_model, validation_dl)
            train_acc, train_time = test(pending_model, train_dl)
            test_acc, test_time = test(pending_model, test_dl)
            val_acc *= 100
            train_acc *= 100
            test_acc *= 100
            print(f"{epoch:^7} | {avg_loss:^12.6f}  {train_acc:^9.2f} |  {test_acc:^9.2f} |  {val_acc:^9.4f} | {epoch_time:^9.2f}")
            log_file.write(f"{epoch:^7} | {avg_loss:^12.6f}  {train_acc:^9.2f} |  {test_acc:^9.2f} |  {val_acc:^9.4f} | {epoch_time:^9.2f}\n")
            log_file.flush()
                
            if avg_loss < min_loss:
                min_loss = avg_loss
                num_no_imp = 0
            else:
                num_no_imp += 1
                
            if num_no_imp > MAX_NO_IMP:
                print('early stop exit')
                log_file.write('\tEarly Stop exit\n')
                log_file.flush()
                break
            
            if epoch < config[MIN_VALID_EPOCHS]:
                continue
            
            if avg_loss > MAX_VALID_LOSS:
                continue
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                optimal_model = copy.deepcopy(pending_model)
                best_epoch = epoch

            if test_acc > best_test_acc:
              best_test_acc = test_acc
              best_test_epoch = epoch
        
        print('train_lstm_nlp - end')
        print("best val: acc={:.3f}".format(best_val_acc) + ", epoch=" + str(best_epoch))
        print("best test: acc={:.3f}".format(best_test_acc) + ", epoch=" + str(best_test_epoch))
        end_train(pending_model, optimal_model, test_dl, validation_dl, config, log_file)
        return pending_model, optimal_model, best_epoch


In [ ]:
with open("workshop/nb_config_lstm.json", "r", encoding="utf-8") as f:
  actual_config = json.load(f)

trainer = Trainer()
last_model, opt_model, best_epoch = trainer.train(actual_config)
print(best_epoch)

latm trainer - start
Freeze embedding matrix = 1
load embedding model
drive/MyDrive/ColabData/embed.model
85899
85898 256


100%|██████████| 85898/85898 [00:08<00:00, 9912.34it/s] 


	w2v after padding: (85900, 256)
generate embedding tensor
start training loops. #epochs = 40
 Epoch  |  Train Loss  |  Train Acc  | Test Acc  |  Val Acc  |  Elapsed 
--------------------------------------------------
   1    |   0.446636      86.76   |    86.43   |   85.5792  |  112.06  
   2    |   0.320785      89.55   |    89.21   |   87.6281  |  111.15  
   3    |   0.290634      90.90   |    90.09   |   88.5737  |  111.65  
   4    |   0.265796      91.77   |    91.01   |   88.8889  |  112.01  
   5    |   0.247293      92.46   |    91.27   |   89.3617  |  110.36  
   6    |   0.237611      93.01   |    91.40   |   90.1497  |  111.86  
   7    |   0.226507      93.70   |    91.86   |   90.6225  |  111.21  
   8    |   0.221044      92.30   |    90.38   |   89.4405  |  111.94  
   9    |   0.257943      93.77   |    91.68   |   90.7801  |  110.35  
  10    |   0.200914      94.51   |    92.15   |   91.4106  |  112.23  
  11    |   0.188575      94.74   |    91.69   |   90.1497  | 

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
  result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,


Optimal Model	Test=test: 0.92444	Val=val: 0.92199
28
